# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [5]:
# Create our features
X = df.drop(columns = ['loan_status'])
X = pd.get_dummies(X, columns = ['home_ownership', 'verification_status', 'pymnt_plan', 'initial_list_status', 'hardship_flag', 'debt_settlement_flag', 'application_type'])
X['next_pymnt_d'] = pd.to_datetime(X['next_pymnt_d'])
X['issue_d'] = pd.to_datetime(X['issue_d'])
# Create our target
y = df.loan_status.to_frame()

In [6]:
X['issue_d'] = X.issue_d.dt.month
X['next_pymnt_d'] = X.next_pymnt_d.dt.month
X = pd.get_dummies(X, columns = ['issue_d', 'next_pymnt_d'])

In [8]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,initial_list_status_w,hardship_flag_N,debt_settlement_flag_N,application_type_Individual,application_type_Joint App,issue_d_1,issue_d_2,issue_d_3,next_pymnt_d_4,next_pymnt_d_5
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.876121,1.0,1.0,0.860340,0.139660,0.451066,0.371696,0.177238,0.383161,0.616839
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.329446,0.0,0.0,0.346637,0.346637,0.497603,0.483261,0.381873,0.486161,0.486161
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,1.000000,1.0,1.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,1.000000,1.0,1.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.000000,1.0,1.0,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
# Check the balance of our target values
Counter(y.loan_status)

Counter({'low_risk': 68470, 'high_risk': 347})

In [10]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

display(y_train.value_counts())

loan_status
low_risk       45891
high_risk        216
dtype: int64

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [11]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [12]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_train_info = scaler.fit(X_train)

In [13]:
# Scale the training and testing data
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [37]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
clf = BalancedRandomForestClassifier(random_state=0)
model = clf.fit(X_train, y_train) 

In [38]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7570861460473473

In [39]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[   85,    46],
       [ 3041, 19538]])

In [40]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.65      0.87      0.05      0.75      0.55       131
   low_risk       1.00      0.87      0.65      0.93      0.75      0.57     22579

avg / total       0.99      0.86      0.65      0.92      0.75      0.57     22710



In [41]:
# List the features sorted in descending order by feature importance
pd.DataFrame(sorted(zip(model.feature_importances_, X.columns), reverse=True), columns = ['score', 'feature']).head(11)

,score,feature
0,0.091155,total_rec_prncp
1,0.059064,last_pymnt_amnt
2,0.056033,total_pymnt_inv
3,0.050831,total_pymnt
4,0.050117,total_rec_int
5,0.029619,int_rate
6,0.018243,tot_hi_cred_lim
7,0.016446,total_bc_limit
8,0.016237,mo_sin_old_il_acct
9,0.016105,annual_inc


### Easy Ensemble Classifier

In [32]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier 
clf = EasyEnsembleClassifier(random_state=0)
model = clf.fit(X_train, y_train) 

In [33]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9039288347714842

In [34]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  116,    15],
       [ 1753, 20826]])

In [35]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.06      0.89      0.92      0.12      0.90      0.81       131
   low_risk       1.00      0.92      0.89      0.96      0.90      0.82     22579

avg / total       0.99      0.92      0.89      0.95      0.90      0.82     22710



### Final Questions

1. Which model had the best balanced accuracy score?

    Easy ensemble classifier

2. Which model had the best recall score?

    Easy ensemble classifier

3. Which model had the best geometric mean score?

    Easy ensemble classifier

4. What are the top three features?

    total_rec_prncp, last_pymnt_amnt, total_pymnt_inv